In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import pandas as pd

import requests
from bs4 import BeautifulSoup
import time

# 목록별 오퍼레이션 확인 완성코드(함수)
- id, pw, operation 개수, 목록 개수를 입력하면 자동으로 업무에 필요한 내용을 크롤링 함
- 단순히 입력하는 업무이기에 코드를 짜놓으면 앞으로 편하게 크롤링 할 수 있을 것이라 판단해 코드를 작성함
- 체계적으로 코드를 구상하지 못해 다소 비효율적으로 코드를 짬
    - 오퍼레이션에 따른 subject5를 다 크롤링 한 뒤에 subject3를 크롤링하면 오퍼레이션에 따른 비교가 어려울 것으로 판단해 페이지를 번갈아가면서 subject3, 5를 수집함
    - 하지만 오퍼레이션에 따른 인덱스를 기록한 뒤 해당 인덱스에 따른 비교를 하면 효율적으로 만들 수 있을 것으로 보임
    - 크롤링 할 때마다 각 항목의 개수를 수정함
    - 다소 비효율적인 방법임
    - 이와 같은 방식 때문에 코드의 길이가 비효율적으로 길어짐
    - 코드를 수정할 수 있지만 크롤링하는 데이터의 양이 그리 많지 않아서 수정하는 시간에 비해 크롤링에 줄어드는 시간이 다소 적을 것으로 생각되기에 코드 수정을 하지 않음
- 해당 과제가 3번 이상 주어진다면 코드 작성에 들인 인풋 대비 아웃풋이 더 높을 것으로 판단

In [2]:
# 입력변수 id, pw, 각 목록에 따른 오퍼레이션 개수, 목록 개수
def Operation(id, pw, operNums, cateNums):
    subject1 = []
    subject2 = []
    subject3 = []
    subject4 = []
    subject5 = []
    operations = []
    categories = []
    start_point = 0

    # 공공데이터 포털 접속
    chrome_driver = 'C:/Users/user/Python(Jupyter)/Big data/Crawling/chromedriver.exe'
    driver = webdriver.Chrome(chrome_driver)
    driver.get("https://www.data.go.kr/uim/login/loginView.do")
    driver.find_element_by_name('mberId').send_keys(id)
    driver.find_element_by_name('pswrd').send_keys(pw)
    driver.find_element_by_xpath('//*[@id="loginVo"]/div[2]/div[2]/div[2]/div/div[1]/button').click()
    driver.find_element_by_xpath('/html/body/div[2]/header/div/button').click()
    driver.implicitly_wait(3)
    driver.find_element_by_xpath('/html/body/div[2]/header/div/div/div/div[2]/div/a[2]').click()

    # 찾고자하는 목록 개수만큼 반복 (api 신청 순서대로 크롤링 진행)
    for cateNum in range(cateNums -1, -1, -1):
        # 각 목록에 따른 오퍼레이션 개수만큼 반복
        for operNum in range(operNums[cateNum]):
            try :
                time.sleep(3)
                driver.find_element_by_xpath('/html/body/div[2]/div/div/div[2]/div[3]/ul/li[{}]/div[1]/a/span'.format(cateNum + 1)).click()
                time.sleep(3)
                driver.find_element_by_xpath('/html/body/div[2]/div/div/div[2]/div[4]/table/tbody/tr[{}]/td[5]/button'.format(operNum * 2 + 1)).click()

                # 의도적으로 크롤링 속도 늦추고 크롤링 진행 (크롤링 속도가 빠르면 제대로 크롤링 불가능)
                time.sleep(3)
                html = driver.page_source
                soup = BeautifulSoup(html, 'html.parser')
                operation = soup.select('#contents > div.col-table.view-toggle > table > tbody > tr:nth-child({}) > td:nth-child(3)'.format(operNum * 2 + 1))[0].text
                operations.append(operation)
                subjects = soup.select('#contents > div.col-table.view-toggle > table > tbody > tr.tr-view.on > td > div > div.bg-inner-table > table > tbody > tr')
                for j in range(len(subjects)):
                    subject1.append(subjects[j].select('td:nth-child(1)')[0].text)
                time.sleep(3)
                driver.find_element_by_xpath('/html/body/div[2]/div/div/div[2]/div[4]/table/tbody/tr[{}]/td/div/div[3]/a'.format(operNum * 2 + 2)).click()

                # 탭 전환을 통해 subject 5, 호출 화면 크롤링 진행
                last_tab = driver.window_handles[-1]
                driver.switch_to.window(window_name=last_tab)
                time.sleep(3)
                html2 = driver.page_source
                soup2 = BeautifulSoup(html2, 'html.parser')
                subjects = soup2.select('span[class != "folder-button fold"] ~ .html-tag')
                temp = []
                
                # subject5 항목 중 중복된 값 제거
                for k in range(len(subjects)):
                    if not(subjects[k].text.replace("<", "").replace(">", "").replace("/", "") in temp):
                        temp.append(subjects[k].text.replace("<", "").replace(">", "").replace("/", ""))
                subject5.extend(temp)
                first_tab = driver.window_handles[0]
                driver.switch_to.window(window_name=first_tab)

                # 데이터 프레임 생성을 위한 operations, subject3, subject5 길이 맞추기
                point = len(operations)
                while len(subject1) != len(subject5) or len(operations) != len(subject5):
                    if len(subject1) > len(subject5):
                        subject5.append("-")
                        if len(subject1) > len(operations):
                            operations.append(operations[point - 1])
                    elif len(subject5) > len(subject1):
                        subject1.append("")
                        if len(subject5) > len(operations):
                            operations.append(operations[point - 1])
                    else:
                        operations.append(operations[point - 1])

                # 상세 설명으로의 이동 오류로 인한 send_keys 함수 사용(그 외엔 오류)
                time.sleep(3)
                driver.find_element_by_xpath('/html/body/div[2]/div/div/div[2]/div[1]/table/tbody/tr[1]/td/button').send_keys(Keys.ENTER)
                time.sleep(1)

                # 현재 할당된 오퍼레이션 넘버에 따른 subject3 크롤링
                driver.find_element_by_xpath('/html/body/div[2]/div/div/div/div[2]/div[3]/div[1]/div/div[1]/select/option[{}]'.format(operNum + 1)).click()
                driver.find_element_by_xpath('/html/body/div[2]/div/div/div/div[2]/div[3]/div[1]/div/div[1]/button').click()
                time.sleep(3)
                html2 = driver.page_source
                soup = BeautifulSoup(html2, 'html.parser')
                subjects = soup.select('#open-api-detail-result > div:nth-child(6) > table > tbody > tr > td:nth-child(2)')
                for a in range(len(subjects)):
                    subject3.append(subjects[a].text)

                # 다른 항목과 subject3의 길이 맞추기
                while len(operations) != len(subject3):
                    if len(operations) > len(subject3):
                        subject3.append('-')
                    else:
                        operations.append(operations[point - 1])
                        subject1.append('')
                        subject5.append('-')

                # 카테고리 이름 크롤링
                category = soup.select('#contents > div.data-search-view > div.data-set-title.open-api > div.tit-area > p')[0].text

                # 카테고리, subject2, subject4 등의 길이 맞추기
                while len(operations) != len(categories):
                    categories.append(category)
                    subject2.append("")
                    subject4.append("")     


                # subject3, subject5의 각 레코드에 대한 순서 맞추기(버블 정렬 참고)
                # o : while 문 반복 횟수 변수
                # idx : 점검할 subject3, subject5의 인덱스

                end_point = len(operations)
                o = end_point - start_point
                idx = -1
                sub3 = subject3[start_point : end_point].copy()
                sub5 = subject5[start_point : end_point].copy()
                while o != 0:
                    o -= 1
                    idx += 1

                    # subject3와 subject5의 레코드가 다를 때 참
                    if not(sub3[idx] == sub5[idx]):

                        # subject3의 레코드가 subject5에 존재하지 않을 때 참
                        if not(sub3[idx] in sub5):

                            # subject5의 레코드가 subject3에 존재하지 않을 때 참
                            # 즉 subject3와 subject5의 각 레코드가 서로에게 없을 때 참
                            if not(sub5[idx] in sub3):

                                # subject3, subject5에 '-' 추가해주고 subject5랑 바꾸기
                                sub3.append('-')
                                sub5.append('-')
                                sub5[idx], sub5[-1] = sub5[-1], sub5[idx]

                                # 레코드 추가에 따른 찾는 범위 재설정 및 반복 횟수 추가
                                end_point += 1 
                                o += 1

                            # subject5의 레코드가 subject3에 존재하지만 '-'이 아닐 때 참
                            elif not(sub5[idx] == '-'):

                                # subject3에서 해당 부분을 바꾸기
                                sub3[idx], sub3[-1] = sub3[-1], sub3[idx]

                        # subject3의 레코드가 subject5에 존재하고 '-'이 아닐 때 참
                        elif not(sub3[idx] == '-'):

                            # subject5에서 해당 부분을 바꾸기
                            temp_idx = sub5.index(sub3[idx])
                            sub5[idx], sub5[temp_idx] = sub3[idx], sub5[idx]


                    # subject3와 subject5가 같지만 둘 다 '-'일 때 참
                    #elif sub3[idx] == '-':

                        # 쓸모 없는 레코드이므로 두 레코드 삭제
                        #print(sub3, idx)
                        #del sub3[idx]
                        #del sub5[idx]
                chk = 0
                while len(sub3) > chk:
                    if sub3[chk] == sub5[chk] and sub3[chk] == '-':
                        del sub3[chk]
                        del sub5[chk]
                        chk += 1
                    else:
                        chk += 1

                subject3[start_point :] = sub3
                subject5[start_point :] = sub5
                        
                # 추가 혹은 삭제된 레코드에 따라 각 항목 개수 조정
                while len(subject3) != len(categories):

                    # subject3와 subject5 추가 시 참(다른 항목에 빈 셀 추가)
                    if len(subject3) > len(categories):
                        categories.append(category)
                        operations.append(operation)
                        subject1.append("")
                        subject2.append("")
                        subject4.append("") 

                    # subject3와 subject5 삭제 시 참(다른 항목의 빈 셀 제거)
                    else:
                        categories = categories[:-1]
                        operations = operations[:-1]
                        subject1 = subject1[:-1]
                        subject2 = subject2[:-1]
                        subject4 = subject4[:-1]

                # 현재 길이를 기준으로 start_point 재 지정
                start_point = len(operations)      

                time.sleep(3)
                driver.find_element_by_xpath('/html/body/div[2]/header/div/button').click()
                time.sleep(3)
                driver.find_element_by_xpath('/html/body/div[2]/header/div/div/div/div[2]/div/a[2]').click()
            except Exception as e:
                driver.quit()
                print('오류가 발생해 함수를 재호출합니다.', e)
                Operation(id, pw, operNums, cateNums)

    # 크롤링한 것을 바탕으로 데이터 프레임 만들기
    df = pd.DataFrame([])
    lists = [categories, operations, subject1, subject2, subject3, subject4, subject5]
    for i in range(len(lists)):
        df[i] = lists[i]
    df.to_excel('C:\\Users\\user\\Desktop\\Work\\Operation.xlsx', index = False, encoding='euc-kr')

In [3]:
id = 'ID'
pw = 'PW'
operNums = [1, 2, 1]
cateNums = 1
Operation(id, pw, operNums, cateNums)

# 완성코드

In [79]:
subject1 = []
subject2 = []
subject3 = []
subject4 = []
subject5 = []
operations = []
categories = []
start_point = 0

# 입력변수 id, pw, 각 목록에 따른 오퍼레이션 개수, 목록 개수
id = 'ID'
pw = 'PW!'
operNums = [1, 2, 1]
cateNums = 3

# 공공데이터 포털 접속
chrome_driver = 'C:/Users/user/Python(Jupyter)/Big data/Crawling/chromedriver.exe'
driver = webdriver.Chrome(chrome_driver)
driver.get("https://www.data.go.kr/uim/login/loginView.do")
driver.find_element_by_name('mberId').send_keys(id)
driver.find_element_by_name('pswrd').send_keys(pw)
driver.find_element_by_xpath('//*[@id="loginVo"]/div[2]/div[2]/div[2]/div/div[1]/button').click()
driver.find_element_by_xpath('/html/body/div[2]/header/div/button').click()
driver.implicitly_wait(3)
driver.find_element_by_xpath('/html/body/div[2]/header/div/div/div/div[2]/div/a[2]').click()

# 찾고자하는 목록 개수만큼 반복 (api 신청 순서대로 크롤링 진행)
for cateNum in range(cateNums -1, -1, -1):
    # 각 목록에 따른 오퍼레이션 개수만큼 반복
    for operNum in range(operNums[cateNum]):
        try :
            time.sleep(3)
            driver.find_element_by_xpath('/html/body/div[2]/div/div/div[2]/div[3]/ul/li[{}]/div[1]/a/span'.format(cateNum + 1)).click()
            time.sleep(3)
            driver.find_element_by_xpath('/html/body/div[2]/div/div/div[2]/div[4]/table/tbody/tr[{}]/td[5]/button'.format(operNum * 2 + 1)).click()
            
            # 의도적으로 크롤링 속도 늦추고 크롤링 진행 (크롤링 속도가 빠르면 제대로 크롤링 불가능)
            time.sleep(3)
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')
            operation = soup.select('#contents > div.col-table.view-toggle > table > tbody > tr:nth-child({}) > td:nth-child(3)'.format(operNum * 2 + 1))[0].text
            operations.append(operation)
            subjects = soup.select('#contents > div.col-table.view-toggle > table > tbody > tr.tr-view.on > td > div > div.bg-inner-table > table > tbody > tr')
            for j in range(len(subjects)):
                subject1.append(subjects[j].select('td:nth-child(1)')[0].text)
            time.sleep(3)
            driver.find_element_by_xpath('/html/body/div[2]/div/div/div[2]/div[4]/table/tbody/tr[{}]/td/div/div[3]/a'.format(operNum * 2 + 2)).click()
            
            # 탭 전환을 통해 subject 5, 호출 화면 크롤링 진행
            last_tab = driver.window_handles[-1]
            driver.switch_to.window(window_name=last_tab)
            time.sleep(3)
            html2 = driver.page_source
            soup2 = BeautifulSoup(html2, 'html.parser')
            subjects = soup2.select('span[class != "folder-button fold"] ~ .html-tag')
            temp = []
            for k in range(len(subjects)):
                if not(subjects[k].text.replace("<", "").replace(">", "").replace("/", "") in temp):
                    temp.append(subjects[k].text.replace("<", "").replace(">", "").replace("/", ""))
            subject5.extend(temp)
            first_tab = driver.window_handles[0]
            driver.switch_to.window(window_name=first_tab)
            
            # 데이터 프레임 생성을 위한 operations, subject3, subject5 길이 맞추기
            point = len(operations)
            while len(subject1) != len(subject5) or len(operations) != len(subject5):
                if len(subject1) > len(subject5):
                    subject5.append("-")
                    if len(subject1) > len(operations):
                        operations.append(operations[point - 1])
                elif len(subject5) > len(subject1):
                    subject1.append("-")
                    if len(subject5) > len(operations):
                        operations.append(operations[point - 1])
                else:
                    operations.append(operations[point - 1])
                    
            # 상세 설명으로의 이동 오류로 인한 send_keys 함수 사용(그 외엔 오류)
            time.sleep(3)
            driver.find_element_by_xpath('/html/body/div[2]/div/div/div[2]/div[1]/table/tbody/tr[1]/td/button').send_keys(Keys.ENTER)
            time.sleep(1)
            
            # 현재 할당된 오퍼레이션 넘버에 따른 subject3 크롤링
            driver.find_element_by_xpath('/html/body/div[2]/div/div/div/div[2]/div[3]/div[1]/div/div[1]/select/option[{}]'.format(operNum + 1)).click()
            driver.find_element_by_xpath('/html/body/div[2]/div/div/div/div[2]/div[3]/div[1]/div/div[1]/button').click()
            time.sleep(3)
            html2 = driver.page_source
            soup = BeautifulSoup(html2, 'html.parser')
            subjects = soup.select('#open-api-detail-result > div:nth-child(6) > table > tbody > tr > td:nth-child(2)')
            for a in range(len(subjects)):
                subject3.append(subjects[a].text)
            
            # 다른 항목과 subject3의 길이 맞추기
            while len(operations) != len(subject3):
                if len(operations) > len(subject3):
                    subject3.append('-')
                else:
                    operations.append(operations[point - 1])
                    subject1.append('')
                    subject5.append('-')
            
            # 카테고리 이름 크롤링
            category = soup.select('#contents > div.data-search-view > div.data-set-title.open-api > div.tit-area > p')[0].text
            
            # 카테고리, subject2, subject4 등의 길이 맞추기
            while len(operations) != len(categories):
                categories.append(category)
                subject2.append("")
                subject4.append("")     
            
            
            # subject3, subject5의 각 레코드에 대한 순서 맞추기(버블 정렬 참고)
            # o : while 문 반복 횟수 변수
            # idx : 점검할 subject3, subject5의 인덱스
            
            end_point = len(operations)
            o = end_point - start_point
            idx = -1
            sub3 = subject3[start_point : end_point].copy()
            sub5 = subject5[start_point : end_point].copy()
            while o != 0:
                o -= 1
                idx += 1
                
                # subject3와 subject5의 레코드가 다를 때 참
                if not(sub3[idx] == sub5[idx]):
                    
                    # subject3의 레코드가 subject5에 존재하지 않을 때 참
                    if not(sub3[idx] in sub5):
                        
                        # subject5의 레코드가 subject3에 존재하지 않을 때 참
                        # 즉 subject3와 subject5의 각 레코드가 서로에게 없을 때 참
                        if not(sub5[idx] in sub3):
                            
                            # subject3, subject5에 '-' 추가해주고 subject5랑 바꾸기
                            sub3.append('-')
                            sub5.append('-')
                            sub5[idx], sub5[-1] = sub5[-1], sub5[idx]
                            
                            # 레코드 추가에 따른 찾는 범위 재설정 및 반복 횟수 추가
                            end_point += 1 
                            o += 1
                        
                        # subject5의 레코드가 subject3에 존재하지만 '-'이 아닐 때 참
                        elif not(sub5[idx] == '-'):
                            
                            # subject3에서 해당 부분을 바꾸기
                            sub3[idx], sub3[-1] = sub3[-1], sub3[idx]
                            
                    # subject3의 레코드가 subject5에 존재하고 '-'이 아닐 때 참
                    elif not(sub3[idx] == '-'):
                        
                        # subject5에서 해당 부분을 바꾸기
                        temp_idx = sub5.index(sub3[idx])
                        sub5[idx], sub5[temp_idx] = sub3[idx], sub5[idx]
                        
                        
                # subject3와 subject5가 같지만 둘 다 '-'일 때 참
                #elif sub3[idx] == '-':
                    
                    # 쓸모 없는 레코드이므로 두 레코드 삭제
                    #print(sub3, idx)
                    #del sub3[idx]
                    #del sub5[idx]
            chk = 0
            while len(sub3) > chk:
                if sub3[chk] == sub5[chk] and sub3[chk] == '-':
                    del sub3[chk]
                    del sub5[chk]
                    chk += 1
                else:
                    chk += 1
                
            subject3[start_point :] = sub3
            subject5[start_point :] = sub5
            
            
            # 추가 혹은 삭제된 레코드에 따라 각 항목 개수 조정
            while len(subject3) != len(categories):
                
                # subject3와 subject5 추가 시 참(다른 항목에 빈 셀 추가)
                if len(subject3) > len(categories):
                    categories.append(category)
                    operations.append(operation)
                    subject1.append("")
                    subject2.append("")
                    subject4.append("") 
                    
                # subject3와 subject5 삭제 시 참(다른 항목에 - 1)
                else:
                    categories = categories[:-1]
                    operations = operations[:-1]
                    subject1 = subject1[:-1]
                    subject2 = subject2[:-1]
                    subject4 = subject4[:-1]
            # 현재 길이를 기준으로 start_point 재 지정
            start_point = len(operations)      
            time.sleep(3)
            driver.find_element_by_xpath('/html/body/div[2]/header/div/button').click()
            time.sleep(3)
            driver.find_element_by_xpath('/html/body/div[2]/header/div/div/div/div[2]/div/a[2]').click()
        except Exception as e:
            print('오류 발생', e)
            driver.quit()
            
# 크롤링한 것을 바탕으로 데이터 프레임 만들기
df = pd.DataFrame([])
lists = [categories, operations, subject1, subject2, subject3, subject4, subject5]
for i in range(len(lists)):
    df[i] = lists[i]
df.to_excel('C:\\Users\\user\\Desktop\\Work\\Operation.xlsx', index = False, encoding='euc-kr')

17 17
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
지점1
지점2
지점3
지점4
29 29
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
지점1
지점2
지점3
지점4
28 28
0
1
2
3
4
5
6
7
8
['resultCode', 'resultMsg', 'ord_cd', 'lec_id', 'lec_title', 'tch_name', 'impl_start_dt', 'impl_finish_dt', 'start_time', '-', 'end_time', 'lec_abstract', 'lec_type', 'day_week', 'impl_place', 'lec_fixedcnt', 'lec_cost', 'ins_name', 'impl_info_tel', 'impl_reg_start', 'impl_reg_finish', 'receipt', 'lec_refer_url', 'ins_hrd_yn', 'lec_cb_yn', 'ins_all_yn', 'lec_terget_name', '-'] 9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
지점1
지점2
subject추가
지점3
지점4
15 15
0
1
2
3
4
5
6
7
8
9
10
11
12
13
['resultCode', 'resultMsg', 'pageIndex', 'firstIndex', 'totCnt', 'dataType', 'industryName', 'addr', 'scale', 'cost', 'period', 'comNum', 'industryHope', 'date', '-'] 14
지점1
지점2
지점3
지점4


In [78]:
len(sub3)

17

In [72]:
            len(sub3)
            while len(sub3) != chk:
                if sub3[chk] == sub5[chk] and sub3[chk] == '-':
                    print(sub3, chk)
                    del sub3[chk]
                    del sub5[chk]
                    chk += 1
                else:
                    print(chk)
                    chk += 1

In [59]:
            i = 0
            for chk in range(len(sub3)):
                if sub3[chk] == sub5[chk] and sub3[chk] == '-':
                    print(sub3, chk)
                    del sub3[chk]
                    del sub5[chk]
                    print(sub3)
                else:
                    i += 1
                    print(i)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
['resultCode', 'resultMsg', 'ord_cd', 'lec_id', 'lec_title', 'tch_name', 'impl_start_dt', 'impl_finish_dt', 'start_time', 'end_time', 'lec_abstract', 'lec_type', 'day_week', 'impl_place', 'lec_fixedcnt', 'lec_cost', 'ins_name', 'impl_info_tel', 'impl_reg_start', 'impl_reg_finish', 'receipt', 'lec_refer_url', 'ins_hrd_yn', 'lec_cb_yn', 'ins_all_yn', 'lec_terget_name', '-', '-'] 27
['resultCode', 'resultMsg', 'ord_cd', 'lec_id', 'lec_title', 'tch_name', 'impl_start_dt', 'impl_finish_dt', 'start_time', 'end_time', 'lec_abstract', 'lec_type', 'day_week', 'impl_place', 'lec_fixedcnt', 'lec_cost', 'ins_name', 'impl_info_tel', 'impl_reg_start', 'impl_reg_finish', 'receipt', 'lec_refer_url', 'ins_hrd_yn', 'lec_cb_yn', 'ins_all_yn', 'lec_terget_name', '-']


In [261]:
start_point = 17
o = end_point - start_point
idx = start_point - 1
subject5 = ['resultCode',
'resultMsg',
'No',
'AC_YEAR',
'SIDO_NAME',
'ENT_NAME',
'USER_NAME',
'TITLE',
'CONTENT',
'FILENAME1',
'FILENAME2',
'FILENAME3',
'LINK',
'REG_DATE',
'numOfRows',
'pageNo',
'totalCount',
'resultCode',
'resultMsg',
'ord_cd',
'lec_id',
'lec_title',
'tch_name',
'impl_start_dt',
'impl_finish_dt',
'start_time',
'end_time',
'lec_abstract',
'lec_target_name',
'lec_type',
'day_week',
'impl_place',
'lec_fixedcnt',
'lec_cost',
'ins_name',
'impl_info_tel',
'impl_reg_start',
'impl_reg_finish',
'receipt',
'lec_refer_url',
'ins_hrd_yn',
'lec_cb_yn',
'ins_all_yn',
'pageNo',
'numOfRows',
'totalCount']
end_point = len(subject5)
sub3 = subject3[start_point : end_point].copy()
end_point, start_point, len(subject3), len(subject5[start_point : end_point])

(46, 17, 29, 29)

In [17]:
start_point = 17
end_point = 46
subject5 = ['resultCode',
'resultMsg',
'No',
'AC_YEAR',
'SIDO_NAME',
'ENT_NAME',
'USER_NAME',
'TITLE',
'CONTENT',
'FILENAME1',
'FILENAME2',
'FILENAME3',
'LINK',
'REG_DATE',
'numOfRows',
'pageNo',
'totalCount',
'resultCode',
'resultMsg',
'ord_cd',
'lec_id',
'lec_title',
'tch_name',
'impl_start_dt',
'impl_finish_dt',
'start_time',
'end_time',
'lec_abstract',
'lec_target_name',
'lec_type',
'day_week',
'impl_place',
'lec_fixedcnt',
'lec_cost',
'ins_name',
'impl_info_tel',
'impl_reg_start',
'impl_reg_finish',
'receipt',
'lec_refer_url',
'ins_hrd_yn',
'lec_cb_yn',
'ins_all_yn',
'pageNo',
'numOfRows',
'totalCount']

In [30]:
            start_point = 17
            end_point = 46
            subject5 = ['resultCode',
            'resultMsg',
            'No',
            'AC_YEAR',
            'SIDO_NAME',
            'ENT_NAME',
            'USER_NAME',
            'TITLE',
            'CONTENT',
            'FILENAME1',
            'FILENAME2',
            'FILENAME3',
            'LINK',
            'REG_DATE',
            'numOfRows',
            'pageNo',
            'totalCount',
            'resultCode',
            'resultMsg',
            'ord_cd',
            'lec_id',
            'lec_title',
            'tch_name',
            'impl_start_dt',
            'impl_finish_dt',
            'start_time',
            'end_time',
            'lec_abstract',
            'lec_target_name',
            'lec_type',
            'day_week',
            'impl_place',
            'lec_fixedcnt',
            'lec_cost',
            'ins_name',
            'impl_info_tel',
            'impl_reg_start',
            'impl_reg_finish',
            'receipt',
            'lec_refer_url',
            'ins_hrd_yn',
            'lec_cb_yn',
            'ins_all_yn',
            'pageNo',
            'numOfRows',
            'totalCount']            

            o = end_point - start_point
            idx = -1
            sub3 = subject3[start_point : end_point].copy()
            sub5 = subject5[start_point : end_point].copy()
            while o != 0:
                o -= 1
                idx += 1
                
                # subject3와 subject5의 레코드가 다를 때 참
                if not(sub3[idx] == sub5[idx]):
                    
                    # subject3의 레코드가 subject5에 존재하지 않을 때 참
                    if not(sub3[idx] in sub5):
                        
                        # subject5의 레코드가 subject3에 존재하지 않을 때 참
                        # 즉 subject3와 subject5의 각 레코드가 서로에게 없을 때 참
                        if not(sub5[idx] in sub3):
                            
                            # subject3, subject5에 '-' 추가해주고 subject5랑 바꾸기
                            sub3.append('-')
                            sub5.append('-')
                            sub5[idx], sub5[-1] = sub5[-1], sub5[idx]
                            
                            # 레코드 추가에 따른 찾는 범위 재설정 및 반복 횟수 추가
                            end_point += 1 
                            o += 1
                        
                        # subject5의 레코드가 subject3에 존재하지만 '-'이 아닐 때 참
                        elif not(sub5[idx] == '-'):
                            
                            # subject3에서 해당 부분을 바꾸기
                            sub3[idx], sub3[-1] = sub3[-1], sub3[idx]
                            
                    # subject3의 레코드가 subject5에 존재하고 '-'이 아닐 때 참
                    elif not(sub3[idx] == '-'):
                        
                        # subject5에서 해당 부분을 바꾸기
                        print(sub5[idx], sub5[sub5.index(sub3[idx])])
                        temp_idx = sub5.index(sub3[idx])
                        sub5[idx], sub5[temp_idx] = sub3[idx], sub5[idx]
                        print(sub5[idx], sub5[temp_idx])
            subject3[start_point :] = sub3
            subject5[start_point :] = sub5


ord_cd numOfRows
numOfRows ord_cd
lec_id pageNo
pageNo lec_id
lec_title totalCount
totalCount lec_title
tch_name ins_all_yn
ins_all_yn tch_name
impl_start_dt ord_cd
ord_cd impl_start_dt
impl_finish_dt lec_id
lec_id impl_finish_dt
start_time tch_name
tch_name start_time
end_time impl_start_dt
impl_start_dt end_time
lec_abstract impl_finish_dt
impl_finish_dt lec_abstract
lec_target_name start_time
start_time lec_target_name
lec_type end_time
end_time lec_type
day_week lec_abstract
lec_abstract day_week
impl_place lec_target_name
lec_target_name impl_place
lec_fixedcnt lec_type
lec_type lec_fixedcnt
lec_cost day_week
day_week lec_cost
ins_name impl_place
impl_place ins_name
impl_info_tel lec_fixedcnt
lec_fixedcnt impl_info_tel
impl_reg_start lec_cost
lec_cost impl_reg_start
impl_reg_finish ins_name
ins_name impl_reg_finish
receipt impl_info_tel
impl_info_tel receipt
lec_refer_url impl_reg_start
impl_reg_start lec_refer_url
ins_hrd_yn impl_reg_finish
impl_reg_finish ins_hrd_yn
lec_cb_yn re

In [23]:
print( subject5[17:])

['resultCode', 'resultMsg', 'numOfRows', 'ins_all_yn', 'totalCount', 'pageNo', 'ord_cd', 'lec_id', 'tch_name', 'impl_start_dt', 'impl_finish_dt', 'start_time', 'end_time', 'lec_abstract', 'lec_target_name', 'lec_type', 'day_week', 'impl_place', 'lec_fixedcnt', 'lec_cost', 'ins_name', 'impl_info_tel', 'impl_reg_start', 'impl_reg_finish', 'receipt', 'lec_refer_url', 'ins_hrd_yn', 'lec_cb_yn', 'lec_title']


In [212]:
sub3 = [1, 2, 3]
sub4 = sub3.copy()
sub3[2], sub3[1] = sub3[1], sub3[2]
sub4[1:] = sub3
sub4

[1, 1, 3, 2]

In [208]:
idx = 4
print(subject3[idx])
print(subject5[idx], subject5[subject5.index(subject3[idx])], subject5.index(subject3[idx]))
subject5[idx], subject5[subject5.index(subject3[idx])] = subject3[idx], subject5[idx]
print(subject5[idx], subject5[subject5.index(subject3[idx])])

totalCount
lec_title totalCount 28
lec_title totalCount


# 연습

In [120]:
cateNums = 3
for i in range(cateNums -1, -1, -1):
    print(i)

2
1
0


In [16]:
last_tab = driver.window_handles[-1]
driver.switch_to.window(window_name=last_tab)
html2 = driver.page_source
soup2 = BeautifulSoup(html2, 'html.parser')
subjects = soup2.select('span[class != "folder-button fold"] ~ .html-tag')
temp = []
print(subjects)
for k in range(len(subjects)):
    if not(subjects[k].text.replace("<", "").replace(">", "").replace("/", "") in temp):
            temp.append(subjects[k].text.replace("<", "").replace(">", "").replace("/", ""))
            print(temp)

[<span class="html-tag">&lt;/resultCode&gt;</span>, <span class="html-tag">&lt;/resultMsg&gt;</span>]
['resultCode']
['resultCode', 'resultMsg']


In [33]:
start_point = [0]
start_point.extend(end_point[:3])
print(len(subject3), len(subject5), len(operations), start_point, end_point)

subjec3 = subject3.copy()
subjec5 = subject5.copy()

for i in range(len(start_point)):
    o = end_point[i] - start_point[i]
    h = start_point[i]
    while o != 0:
        if not(subjec3[h] == subjec5[h]):
            if not(subjec3[h] in subjec5):
                if not(subjec5[h] in subjec3):
                    subjec3.append('-')
                    subjec5.append('-')
                    subjec5[h], subjec5[-1] = subjec5[-1], subjec5[h]
                    h += 1
                else:
                    subjec3[h], subjec3[-1] = subjec3[-1], subjec3[h]
                    o -= 1
                    h += 1
            else:
                subjec5[h], subjec5[subjec5.index(subjec3[h])] = subjec3[h], subjec5[h]
                o -= 1
                h += 1
        elif subjec3[h] == '-':
            del subjec3[h]
            del subjec5[h]
            o -= 1
            h += 1
        else:
            o -= 1
            h += 1
    print(subjec5[:h], subject3[:h], o, h, sep = '\n', end='\n\n\n')


59 59 59 [0, 17, 35, 47] [17, 35, 47, 59]
['resultCode', 'resultMsg', 'numOfRows', 'pageNo', 'totalCount', 'No', 'AC_YEAR', 'SIDO_NAME', 'ENT_NAME', 'USER_NAME', 'TITLE', 'CONTENT', 'FILENAME1', 'FILENAME2', 'FILENAME3', 'LINK', 'REG_DATE']
['resultCode', 'resultMsg', 'numOfRows', 'pageNo', 'totalCount', 'No', 'AC_YEAR', 'SIDO_NAME', 'ENT_NAME', 'USER_NAME', 'TITLE', 'CONTENT', 'FILENAME1', 'FILENAME2', 'FILENAME3', 'LINK', 'REG_DATE']
0
17


['resultCode', 'resultMsg', 'numOfRows', 'pageNo', 'totalCount', 'No', 'AC_YEAR', 'SIDO_NAME', 'ENT_NAME', 'USER_NAME', 'TITLE', 'CONTENT', 'FILENAME1', 'FILENAME2', 'FILENAME3', 'LINK', 'REG_DATE', 'resultCode', 'resultMsg', 'cartype', 'emisCertiDate', 'emisCertiNo', 'fueltype', 'gubun1', 'gubun2', 'mustard', 'mustard2', 'noiseCertiDate', 'noiseCertiNo', 'officeNm', 'vehNm', 'vehType', 'numOfRows', 'pageNo', 'totalCount']
['resultCode', 'resultMsg', 'numOfRows', 'pageNo', 'totalCount', 'No', 'AC_YEAR', 'SIDO_NAME', 'ENT_NAME', 'USER_NAME', 'TITLE

In [23]:
end_point, start_point

([17, 35, 47, 59], [0, 17, 35, 47])

In [24]:
a = [1, 2, 3, 4, 5, 19, 24]
del a[5]
a

[1, 2, 3, 4, 5, 24]

In [30]:
testdf = pd.DataFrame([])
testdf[1] = subject3
testdf[2] = subject5
testdf

,1,2
0,resultCode,resultCode
1,resultMsg,resultMsg
2,numOfRows,-
3,pageNo,-
4,totalCount,-
5,No,No
6,AC_YEAR,AC_YEAR
7,SIDO_NAME,SIDO_NAME
8,ENT_NAME,ENT_NAME
9,USER_NAME,USER_NAME


In [ ]:
            end_point = len(operations)
            for h in range(start_point, end_point):
                if not(subject3[h] == subject5[h]):
                    if not(subject3[h] in subject5):
                        if not(subject5[h] in subject3):
                            subject3.append('-')
                            subject5.append('-')
                            sub5ject[h], subject5[-1] = subject5[-1], subject5[h]
                        else:
                            subject3[h], subject3[-1] = subject3[-1], subject3[h]
                    else:
                        subject5[h], subject5[subject5.index(subject3[h])] = subject3[h], subject5[h]
            start_point = len(operations)
            

In [110]:
sub3 = subject3.copy()[:20]
sub5 = subject5.copy()[:20]

print(sub3, sub5)

['resultCode', 'resultMsg', 'numOfRows', 'pageNo', 'totalCount', 'No', 'AC_YEAR', 'SIDO_NAME', 'ENT_NAME', 'USER_NAME', 'TITLE', 'CONTENT', 'FILENAME1', 'FILENAME2', 'FILENAME3', 'LINK', 'REG_DATE', 'serviceKey', 'pageNo', 'numOfRows'] ['resultCode', 'resultMsg', 'item', 'emisCertiDate', 'fueltype', 'No', 'AC_YEAR', 'SIDO_NAME', 'ENT_NAME', 'USER_NAME', 'TITLE', 'CONTENT', 'FILENAME1', 'FILENAME2', 'FILENAME3', 'LINK', 'REG_DATE', 'resultCode', 'resultMsg', 'items']


In [151]:
for h in range(len(sub3) - 1):
    idx = h
    for l in range(h + 1, len(sub3)):
        if sub3[l] == sub5[l]:
            break
        sub5[l], sub5[idx] = sub5[idx], sub5[l]
print(sub3, sub5)


['resultCode', 'resultMsg', 'numOfRows', 'pageNo', 'totalCount', 'dataType', 'code', 'areaNo', 'date', 'today', 'tomorrow', 'todayaftertomorrow', 'a', 'a', 'a', 'a', 'a'] ['resultCode', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', 'resultMsg', '-', '-', '-', '-', '-', 'b', 'b', 'a', 'a', 'a']


In [6]:
test_tab = driver.window_handles[1]
driver.switch_to.window(window_name=test_tab)
testhtml = driver.page_source
testsoup = BeautifulSoup(testhtml, 'html.parser')
test = testsoup.select('span[class != "folder-button fold"] ~ .html-tag')
test

[<span class="html-tag">&lt;/resultCode&gt;</span>,
 <span class="html-tag">&lt;/resultMsg&gt;</span>,
 <span class="html-tag">&lt;/No&gt;</span>,
 <span class="html-tag">&lt;/AC_YEAR&gt;</span>,
 <span class="html-tag">&lt;/SIDO_NAME&gt;</span>,
 <span class="html-tag">&lt;/ENT_NAME&gt;</span>,
 <span class="html-tag">&lt;/USER_NAME&gt;</span>,
 <span class="html-tag">&lt;/TITLE&gt;</span>,
 <span class="html-tag">&lt;/CONTENT&gt;</span>,
 <span class="html-tag">&lt;/FILENAME1&gt;</span>,
 <span class="html-tag">&lt;/FILENAME2&gt;</span>,
 <span class="html-tag">&lt;/FILENAME3&gt;</span>,
 <span class="html-tag">&lt;/LINK&gt;</span>,
 <span class="html-tag">&lt;/REG_DATE&gt;</span>,
 <span class="html-tag">&lt;/No&gt;</span>,
 <span class="html-tag">&lt;/AC_YEAR&gt;</span>,
 <span class="html-tag">&lt;/SIDO_NAME&gt;</span>,
 <span class="html-tag">&lt;/ENT_NAME&gt;</span>,
 <span class="html-tag">&lt;/USER_NAME&gt;</span>,
 <span class="html-tag">&lt;/TITLE&gt;</span>,
 <span class="h

In [111]:
for h in range(20):
    if not(sub3[h] == sub5[h]):
        if not(sub3[h] in sub5):
            if not(sub5[h] in sub3):
                sub3.append('-')
                sub5.append('-')
                sub5[h], sub5[-1] = sub5[-1], sub5[h]
            else:
                sub3[h], sub3[-1] = sub3[-1], sub3[h]
        else:
            sub5[h], sub5[sub5.index(sub3[h])] = sub3[h], sub5[h]
print(sub3, sub5)

['resultCode', 'resultMsg', 'numOfRows', 'pageNo', 'totalCount', 'No', 'AC_YEAR', 'SIDO_NAME', 'ENT_NAME', 'USER_NAME', 'TITLE', 'CONTENT', 'FILENAME1', 'FILENAME2', 'FILENAME3', 'LINK', 'REG_DATE', '-', 'serviceKey', 'numOfRows', '-', '-', 'pageNo', '-'] ['resultCode', 'resultMsg', '-', '-', '-', 'No', 'AC_YEAR', 'SIDO_NAME', 'ENT_NAME', 'USER_NAME', 'TITLE', 'CONTENT', 'FILENAME1', 'FILENAME2', 'FILENAME3', 'LINK', 'REG_DATE', 'resultCode', 'resultMsg', '-', 'item', 'emisCertiDate', 'fueltype', 'items']


In [127]:
for h in range(start_point):
    if subject3[h] in subject5:
        if not(subject3[h] == subject5[h]):
            subject5[h], subject5[subject5.index(subject3[h])] = subject3[h], subject5[h]
print(subject5)           

['resultCode', 'resultMsg', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-']


In [109]:
subject5[4]

'fueltype'

In [91]:
for i in range(len(sub3)):
    if (sub3[i] in sub5):
        if not(sub3[i] == sub5[i]):
            sub5[i], sub5[sub5.index(sub3[i])] = sub3[i], sub5[i]
print(sub3)
print(sub5)

['resultCode', 'resultMsg', 'numOfRows', 'pageNo', 'totalCount', 'No', 'AC_YEAR', 'SIDO_NAME', 'ENT_NAME', 'USER_NAME', 'TITLE', 'CONTENT', 'FILENAME1', 'FILENAME2', 'FILENAME3', 'LINK']
['resultCode', 'resultMsg', 'item', '-', 'REG_DATE', 'No', 'AC_YEAR', 'SIDO_NAME', 'ENT_NAME', 'USER_NAME', 'TITLE', 'CONTENT', 'FILENAME1', 'FILENAME2', 'FILENAME3', 'LINK']


In [59]:
df = pd.DataFrame([])
lists = [categories, operations, subject1, subject2, subject3, subject4, subject5]
for i in range(len(lists)):
    df[i] = lists[i]

df.to_excel('C:\\Users\\user\\Desktop\\Work\\Operation.xlsx', index = False, encoding='euc-kr')

In [137]:
subject1 = []
subject5 = []
operations = []
id = 'ID'
pw = 'PW'
cateNum = 3

chrome_driver = 'C:/Users/user/Python(Jupyter)/Big data/Crawling/chromedriver.exe'
driver = webdriver.Chrome(chrome_driver)
driver.get("https://www.data.go.kr/uim/login/loginView.do")
driver.find_element_by_name('mberId').send_keys(id)
driver.find_element_by_name('pswrd').send_keys(pw)
driver.find_element_by_xpath('//*[@id="loginVo"]/div[2]/div[2]/div[2]/div/div[1]/button').click()
driver.find_element_by_xpath('/html/body/div[2]/header/div/button').click()
driver.implicitly_wait(2)
driver.find_element_by_xpath('/html/body/div[2]/header/div/div/div/div[2]/div/a[2]').click()
for h in range(cateNum):
    driver.find_element_by_xpath('/html/body/div[2]/div/div/div[2]/div[3]/ul/li[{}]/div[1]/a/span'.format(h + 1)).click()
    print('해당 부분에 코드 복붙')
    driver.find_element_by_xpath('/html/body/div[2]/div/div/div[2]/div[1]/table/tbody/tr[1]/td/button').click()
    print('해당 부분에 새로운 코드 작성')
    if h + 1 != cateNum:
        driver.find_element_by_xpath('/html/body/div[2]/header/div/button').click()
        time.sleep(1)
        driver.find_element_by_xpath('/html/body/div[2]/header/div/div/div/div[2]/div/a[2]').click()
        print(h)

해당 부분에 코드 복붙
해당 부분에 새로운 코드 작성
0
해당 부분에 코드 복붙
해당 부분에 새로운 코드 작성
1
해당 부분에 코드 복붙
해당 부분에 새로운 코드 작성


In [ ]:
subject3 = []
for b in range(operNum):
    driver.find_element_by_xpath('/html/body/div[2]/div/div/div/div[2]/div[3]/div[1]/div/div[1]/select/option[{}]'.format(b + 1)).click()
    driver.find_element_by_xpath('/html/body/div[2]/div/div/div/div[2]/div[3]/div[1]/div/div[1]/button').click()
    html2 = driver.page_source
    soup = BeautifulSoup(html2, 'html.parser')
    subjects = soup.select('#open-api-detail-result > div:nth-child(6) > table > tbody > tr > td:nth-child(2)')
    for a in range(len(subjects)):
        subject3.append(subjects[a].text)
    while len(operations) != len(subject3):
        if len(operations) > len(subject3):
            subject3.append('-')
        else:
            operations.append(operations[point - 1])
            subject1.append('')
            subject5.append('-')

In [31]:
while len(subject1) != len(subject2) or len(operations) != len(subject1):
    if len(subject1) > len(subject2):
        subject2.append("")
        print(1)
        if len(subject1) > len(operations):
            operations.append(operations[point - 1])
            print(2)
    elif len(subject2) > len(subject1):
        subject1.append("")
        print(3)
        if len(subject2) > len(operations):
            operations.append(operations[point - 1])
            print(4)
    else:
        operations.append(operations[point - 1])

In [ ]:
driver.find_element_by_xpath('/html/body/div[2]/div/div/div[2]/div[1]/table/tbody/tr[1]/td/button')

In [12]:
while len(subject1) != len(subject2) or len(operations) != len(subject1):
    if len(subject1) > len(subject2):
        subject2.append("")
    operations.append(operations[0])
print(operations, subject1, subject2)
df = pd.DataFrame([])
df['operations'] = operations
df

['천식폐질환가능지수조회', '천식폐질환가능지수조회', '천식폐질환가능지수조회', '천식폐질환가능지수조회', '천식폐질환가능지수조회', '천식폐질환가능지수조회'] ['ServiceKey', 'pageNo', 'numOfRows', 'dataType', 'areaNo', 'time'] ['resultCode', 'resultMsg', '', '', '', '']


,operation
0,천식폐질환가능지수조회
1,천식폐질환가능지수조회
2,천식폐질환가능지수조회
3,천식폐질환가능지수조회
4,천식폐질환가능지수조회
5,천식폐질환가능지수조회


In [99]:
subject1 = []
subject2 = []
operations = []
id = 'ID'
pw = 'PW'
cateNum = 3

chrome_driver = 'C:/Users/user/Python(Jupyter)/Big data/Crawling/chromedriver.exe'
driver = webdriver.Chrome(chrome_driver)
driver.get("https://www.data.go.kr/uim/login/loginView.do")
driver.find_element_by_name('mberId').send_keys(id)
driver.find_element_by_name('pswrd').send_keys(pw)
driver.find_element_by_xpath('//*[@id="loginVo"]/div[2]/div[2]/div[2]/div/div[1]/button').click()
driver.find_element_by_xpath('/html/body/div[2]/header/div/button').click()
driver.implicitly_wait(2)
driver.find_element_by_xpath('/html/body/div[2]/header/div/div/div/div[2]/div/a[2]').click()
for h in range(cateNum):
    driver.find_element_by_xpath('/html/body/div[2]/div/div/div[2]/div[3]/ul/li[{}]/div[1]/a/span'.format(h + 1)).click()
    print('해당 부분에 코드 복붙')
    driver.find_element_by_xpath('/html/body/div[2]/div/div/div[2]/div[1]/table/tbody/tr[1]/td/button').click()
    print('해당 부분에 새로운 코드 작성')
    if h + 1 != cateNum:
        driver.find_element_by_xpath('/html/body/div[2]/header/div/button').click()
        time.sleep(1)
        driver.find_element_by_xpath('/html/body/div[2]/header/div/div/div/div[2]/div/a[2]').click()
        print(h)

해당 부분에 코드 복붙
해당 부분에 새로운 코드 작성
0
해당 부분에 코드 복붙
해당 부분에 새로운 코드 작성
1
해당 부분에 코드 복붙
해당 부분에 새로운 코드 작성


In [113]:
cateNum

3

In [114]:
subject3 = []
for b in range(operNum):
    driver.find_element_by_xpath('/html/body/div[2]/div/div/div/div[2]/div[3]/div[1]/div/div[1]/select/option[{}]'.format(b + 1)).click()
    driver.find_element_by_xpath('/html/body/div[2]/div/div/div/div[2]/div[3]/div[1]/div/div[1]/button').click()
    html2 = driver.page_source
    soup = BeautifulSoup(html2, 'html.parser')
    subjects = soup.select('#open-api-detail-result > div:nth-child(6) > table > tbody > tr > td:nth-child(2)')
    for a in range(len(subjects)):
        subject3.append(subjects[a].text)
    while len(operations) != len(subject3):
        if len(operations) > len(subject3):
            subject3.append('-')
        else:
            operations.append('-')
            subject1.append('-')
            subject5.append('-')

NameError: name 'operNum' is not defined

In [176]:
try :
    driver.find_element_by_xpath('/html/body/div[2]/div/div/div[2]/div[4]/table/tbody/tr[30]/td[5]/button')
except Exception:
    print(1)

1


In [163]:
chrome_driver = 'C:/Users/user/Python(Jupyter)/Big data/Crawling/chromedriver.exe'
driver = webdriver.Chrome(chrome_driver)
driver.get("https://www.data.go.kr/uim/login/loginView.do")
driver.find_element_by_name('mberId').send_keys('ID')
driver.find_element_by_name('pswrd').send_keys('PW')
driver.find_element_by_xpath('//*[@id="loginVo"]/div[2]/div[2]/div[2]/div/div[1]/button').click()
driver.implicitly_wait(2)
driver.find_element_by_xpath('/html/body/div[2]/header/div/button').click()
driver.find_element_by_xpath('/html/body/div[2]/header/div/div/div/div[2]/div/a[2]').click()
driver.implicitly_wait(2)
driver.find_element_by_xpath('/html/body/div[2]/div/div/div[2]/div[3]/ul/li[1]/div[1]/a/span').click()
driver.find_element_by_xpath('/html/body/div[2]/div/div/div[2]/div[4]/table/tbody/tr[1]/td[5]/button').click()
time.sleep(2)
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
subjects = soup.select('#contents > div.col-table.view-toggle > table > tbody > tr.tr-view.on > td > div > div.bg-inner-table > table > tbody > tr')
subject1 = []
for i in range(len(subjects)):
    subject1.append(subjects[i].select('td:nth-child(1)')[0].text)
driver.find_element_by_xpath('/html/body/div[2]/div/div/div[2]/div[4]/table/tbody/tr[2]/td/div/div[3]/a').click()
last_tab = driver.window_handles[-1]
driver.switch_to.window(window_name=last_tab)
html2 = driver.page_source
soup2 = BeautifulSoup(html2, 'html.parser')
subjects = soup2.select('.opened .opened .html-tag')
subject2 = []
for i in range(len(subjects)):
    if not(subjects[i].text.replace("<", "").replace(">", "") in subject2):
        if not( '/' in subjects[i].text):
            subject2.append(subjects[i].text.replace("<", "").replace(">", ""))

In [166]:
url = 'https://www.data.go.kr/data/15061205/openapi.do'
response = requests.get(url)
response

<Response [200]>

In [129]:
subjects = soup2.select('.html-tag')
subject2 = []
for i in range(len(subjects)):
    if not(subjects[i].text in subject2):
        if not( '/' in subjects[i].text):
            subject2.append(subjects[i].text)
subject2

['<response>',
 '<reMsg>',
 '<resultCode>',
 '<resultMsg>',
 '<items>',
 '<item>',
 '<No>',
 '<AC_YEAR>',
 '<SIDO_NAME>',
 '<ENT_NAME>',
 '<USER_NAME>',
 '<TITLE>',
 '<CONTENT>',
 '<FILENAME1>',
 '<FILENAME2>',
 '<FILENAME3>',
 '<LINK>',
 '<REG_DATE>',
 '<numOfRows>',
 '<pageNo>',
 '<totalCount>']

#contents > div.col-table.view-toggle > table > tbody > tr.tr-view.on > td > div > div.bg-inner-table > table > tbody > tr:nth-child(1) > td:nth-child(1)

In [56]:
subjects = soup.select('#contents > div.col-table.view-toggle > table > tbody > tr.tr-view.on > td > div > div.bg-inner-table > table > tbody > tr')
subject1 = []

for i in range(len(subjects)):
    subject1.append(subjects[i].select('td:nth-child(1)')[0].text)
subject1

['ServiceKey',
 'pageNo',
 'numOfRows',
 'ac_year',
 'from_month',
 'to_month',
 'sidoCd',
 'type']

In [150]:
chrome_driver = 'C:/Users/user/Python(Jupyter)/Big data/Crawling/chromedriver.exe'
driver = webdriver.Chrome(chrome_driver)
driver.get("https://www.data.go.kr/uim/login/loginView.do")
driver.find_element_by_name('mberId').send_keys('ID')
driver.find_element_by_name('pswrd').send_keys('PW')
driver.find_element_by_xpath('//*[@id="loginVo"]/div[2]/div[2]/div[2]/div/div[1]/button').click()
driver.implicitly_wait(2)
driver.find_element_by_xpath('/html/body/div[2]/header/div/button').click()
driver.find_element_by_xpath('/html/body/div[2]/header/div/div/div/div[2]/div/a[2]').click()
driver.find_element_by_xpath('/html/body/div[2]/div/div/div[2]/div[3]/ul/li[1]/div[1]/a/span').click()
driver.find_element_by_xpath('/html/body/div[2]/div/div/div[2]/div[4]/table/tbody/tr[1]/td[5]/button').click()
time.sleep(2)
driver.find_element_by_xpath('//*[@id="contents"]/div[4]/table/tbody/tr[2]/td/div/div[3]/a').click()
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
soup

<html lang="ko"><head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1.0, user-scalable=yes" name="viewport"/>
<meta content="telephone=no" name="format-detection"/>
<meta content="공공데이터 포털" name="title" property="og:title"/>
<meta content="website" name="type" property="og:type"/>
<meta content="국가에서 보유하고 있는 다양한 데이터를『공공데이터의 제공 및 이용 활성화에 관한 법률(제11956호)』에 따라 개방하여 국민들이 보다 쉽고 용이하게 공유•활용할 수 있도록 공공데이터(Dataset)와 Open API로 제공하는 사이트입니다." name="description" property="og:description"/>
<meta content="http://data.go.kr:8088/images/biz/data-search/bg-nation.png" name="image" property="og:image"/>
<link href="/images/biz/DATA.ico" rel="icon" type="image/x-icon"/>
<meta content="dddd02420b56aeebf384a2b9b68a8d20a3bcd1c7" name="naver-site-verification"/>
<title>개발계정 상세보기 | 공공데이터포털</title>
<!-- css -->
<link href="/css/biz/cmm/tcs/import.css" media="all" rel="stylesheet"/>
<link href="/css/app.css?t=202009273_1752" media=

In [149]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
soup

<html lang="ko"><head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1.0, user-scalable=yes" name="viewport"/>
<meta content="telephone=no" name="format-detection"/>
<meta content="공공데이터 포털" name="title" property="og:title"/>
<meta content="website" name="type" property="og:type"/>
<meta content="국가에서 보유하고 있는 다양한 데이터를『공공데이터의 제공 및 이용 활성화에 관한 법률(제11956호)』에 따라 개방하여 국민들이 보다 쉽고 용이하게 공유•활용할 수 있도록 공공데이터(Dataset)와 Open API로 제공하는 사이트입니다." name="description" property="og:description"/>
<meta content="http://data.go.kr:8088/images/biz/data-search/bg-nation.png" name="image" property="og:image"/>
<link href="/images/biz/DATA.ico" rel="icon" type="image/x-icon"/>
<meta content="dddd02420b56aeebf384a2b9b68a8d20a3bcd1c7" name="naver-site-verification"/>
<title>개발계정 상세보기 | 공공데이터포털</title>
<!-- css -->
<link href="/css/biz/cmm/tcs/import.css" media="all" rel="stylesheet"/>
<link href="/css/app.css?t=202009273_1749" media=